## Guided Project: Answering Business Questions Using SQL

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# Overview of the Data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to purchase

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA, and then make a recommendation for the three artists (among below 4 artists) whose albums we should purchase for the Chinook record store.

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

In [4]:
%%sql

WITH ta AS (
SELECT g.name genre, COUNT(t.track_id) num_track
FROM genre g
LEFT JOIN track t ON g.genre_id = t.genre_id
LEFT JOIN invoice_line il ON il.track_id = t.track_id
LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
WHERE i.billing_country = "USA"
GROUP BY 1),

    pc AS (
SELECT
    genre, 
    num_track,
    ROUND(CAST(num_track AS FLOAT) / (SELECT SUM(num_track) FROM ta) *100,2) num_track_pc
FROM ta)

SELECT *
FROM pc
ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre,num_track,num_track_pc
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


With the above findings, we recommend choosing albums produced by the following three artists: Red Tone (Punk), Slim Jim Bites (Blues), and Meteor and the Girls (Pop)

## Analysing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We'll analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.



In [5]:
%%sql
SELECT e.first_name || " " || e.last_name employee_name, e.reports_to, e.birthdate, e.hire_date, (e.hire_date - e.birthdate) age, ROUND(SUM(i.total)) total_sales
FROM employee e
INNER JOIN customer c
ON e.employee_id = c.support_rep_id
LEFT JOIN invoice i
ON c.customer_id = i.customer_id
GROUP BY 1

 * sqlite:///chinook.db
Done.


employee_name,reports_to,birthdate,hire_date,age,total_sales
Jane Peacock,2,1973-08-29 00:00:00,2017-04-01 00:00:00,44,1732.0
Margaret Park,2,1947-09-19 00:00:00,2017-05-03 00:00:00,70,1584.0
Steve Johnson,2,1965-03-03 00:00:00,2017-10-17 00:00:00,52,1394.0


With the above findings, we see that Jane Peacock is the employee performing better than other sales support agent. The age difference might be indicative of employee performance as Jane was the youngest among the three staff when they joined the company in the same year 2017.

## Analyzing Sales by Country

Next, we'll analyze the sales data for customers from each different country. 

In [6]:
%%sql

WITH t3 AS (select c.country sort, 
COUNT(distinct c.customer_id) customers, 
SUM(i.total) sales, 
COUNT(DISTINCT i.invoice_id) num_inv
from customer c
LEFT JOIN invoice i
ON c.customer_id = i.customer_id
group by 1
order by 2 DESC),


t4 AS (SELECT 
CASE WHEN t3.customers = 1 THEN 'Other'
ELSE t3.sort
END AS country,
t3.customers,
t3.sales,
t3.num_inv
from t3),

t5 AS (SELECT t4.country country, SUM(t4.customers) customers, 
ROUND(SUM(t4.sales),2) total_sales,
ROUND(SUM(t4.sales) / SUM(t4.num_inv),2) avg_order,
ROUND(SUM(t4.sales) / SUM(t4.customers),2) customer_lifetime_value,
CASE WHEN t4.country = 'Other' THEN 1
ELSE 0
END AS count
from t4
group by 1)

SELECT country, 
    customers,
    total_sales,
    avg_order, 
    customer_lifetime_value
FROM t5
ORDER BY count ASC, total_sales DESC

 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.57
Other,15,1094.94,7.45,73.0


## Album vs Individual Tracks

In [39]:
%%sql

WITH t1 AS (
SELECT invoice_id, MIN(track_id) first_track
FROM invoice_line
GROUP BY invoice_id
),    

t2 AS (

SELECT t1.*,

CASE 
    WHEN 
        (
        SELECT t.track_id FROM track t
        WHERE t.album_id = (
                            SELECT album_id FROM track
                            WHERE track_id = t1.first_track
                            )
                            

        EXCEPT
        
        SELECT il.track_id FROM invoice_line il
        WHERE il.invoice_id = t1.invoice_id
        ) IS NULL
        
        AND 
        
        (
        SELECT track_id FROM invoice_line il
        WHERE invoice_id = t1.invoice_id
        
        EXCEPT
        
        SELECT track_id FROM track t
        WHERE album_id = (SELECT album_id
                            FROM track
                           WHERE track_id = t1.first_track)    
        ) IS NULL
        
        THEN "yes"
        ELSE "no"
    END AS "album_purchase"
FROM t1
)

SELECT album_purchase,
       COUNT(album_purchase) number_of_invoices,
       CAST(COUNT(album_purchase) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percentage_of_invoices
FROM t2
GROUP BY album_purchase


 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percentage_of_invoices
no,500,0.8143322475570033
yes,114,0.18566775244299674


From the result, it's clear that the number of single track purchases outweight album purchases (Album purchases account for only 18.6% of purchases). Thus, the Chinook store should not continue to buy full albums from record companies.  